# PVInspect lazy loading

PVInspect supports lazy loading, such that it is not necessary to load the complete data associated to a (possibly long) `ImageSequence` to disk. To demonstrate this, let's first download a bunch of images:

In [1]:
!gdown --id 16O4Wf_aGNuiiUw8qDFYKQgqmtXqEvPx4

/Users/z003v8ys/dev/pvinspect/.venv/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=16O4Wf_aGNuiiUw8qDFYKQgqmtXqEvPx4
From (redirected): https://drive.google.com/uc?id=16O4Wf_aGNuiiUw8qDFYKQgqmtXqEvPx4&confirm=t&uuid=d514de6b-6589-4af9-90a2-a9bd46f5102a
To: /Users/z003v8ys/dev/pvinspect/examples/pvinspect_demo_images.zip
100%|██████████████████████████████████████| 90.7M/90.7M [00:27<00:00, 3.25MB/s]


In [2]:
%%capture
!unzip -o pvinspect_demo_images

Now, let's install the toolbox:

In [3]:
import pvinspect as pv
from pvinspect.data.image import LazyImage
from pathlib import Path
import numpy as np

We can now load the images and specify that they should be lazy loaded by using the `lazy`-argument:

In [4]:
seq = pv.data.io.read_images(Path("images"), lazy=True)
len(seq)

2624

So this essentially loaded 2624 images in no time, already showing the power of lazy loading. We may also verify that this uses lazy loading by inspecting the type:

In [5]:
seq

In contrast, disabling lazy loading gives an `EagerImageSequence` and takes considerably longer:

In [6]:
pv.data.io.read_images(Path("images"), lazy=False)

## Lazy loading sequences

Lazy loading essentially applies to sequences, as well as to images. For sequences it means that only the meta data associated with a sequence is loaded. In any case, this meta data consists of the actual image filenames:

In [7]:
seq.meta

,original_filename
0,cell1922.png
1,cell0382.png
2,cell2595.png
3,cell2581.png
4,cell0396.png
...,...
2619,cell1091.png
2620,cell2598.png
2621,cell1085.png
2622,cell1913.png


Since class instantiation takes some time too, instances of class `LazyImage` are only created when needed. We can verify this by accessing a single image twice:

In [8]:
seq[0]

In [9]:
seq[0]

Here, we see that any access of a single image returns a new instance. Nevertheless, they both refer to the same image:

In [10]:
seq[0].meta

original_filename    cell1922.png
Name: 0, dtype: object

In [11]:
seq[0].meta

original_filename    cell1922.png
Name: 0, dtype: object

## Lazy loading images

As we've seen, accessing a single image from a `LazyImageSequence` provides us with a `LazyImage`. Looking at the implementation of the `LazyImage.data`-attribute, we observe that this does not simply return the data. Instead, it calls a function that loads the data:

```python
@property
    def data(self) -> np.ndarray:
        return self._data.load()
```

Let's investigate the type of this hidden data attribute:

In [12]:
img = seq[0]
img._data

In fact, a lazy image is created by passing in an instance of `LazyImage.LazyData`. Constructing this `LazyImage.LazyData` involves passing a function that is called to load the data. Consider the following example:

In [13]:
def load_fn():
  print("Loading...")
  return np.array([0])

img2 = LazyImage(LazyImage.LazyData(load_fn))

It becomes apparent that `load_fn` is not called for not. But it is, as soon as we access the `.data`-attribute:

In [14]:
img2.data

Loading...


array([0], dtype=int32)

## The complete mechanism

So how does the complete loading mechanism in PVInspect work now. When calling `pv.data.io.read_images(..., lazy=True)`, PVInspect constructs then image meta data which, in any case, contains the `original_filename`-attribute. Then, it instantiates the `LazyImageSequence` and passes the meta data as well as a loading function. This loading function now takes the filename from the meta attribute and uses it, together with the path, to load the data. 

From `pvinspect.data.io.py` we see that this is done by using only a few lines of code:
```python
load_seq_item_fn = lambda x: LazyImage(
    LazyImage.LazyData(lambda: skio.imread(path / x["original_filename"])), x
)
return LazyImageSequence(meta, load_seq_item_fn)
```